# Heat equation

## imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from matplotlib import animation
from IPython.display import HTML
from scipy import special
import math

## initialization

In [2]:
X_max = 100
Y_max = 100
X_min = Y_min = 0

NUMBER_X = 1000
NUMBER_Y = 1000

In [3]:
X_RANGE = np.linspace(X_min, X_max, NUMBER_X)
Y_RANGE = np.linspace(Y_min, Y_max, NUMBER_Y)

X_STEP = X_RANGE[1] - X_RANGE[0]
Y_STEP = Y_RANGE[1] - Y_RANGE[0]

In [4]:
X, Y = np.meshgrid(X_RANGE, Y_RANGE, indexing='ij')

T = np.zeros_like(X)
lambdas = np.zeros_like(X)
source_c = np.zeros_like(X)
source_p = np.zeros_like(X)

## boundary conditions

## main procedure